# Подход №3. Анализ не по словам, а по словосочетаниям. Основа словаря теперь - именно словосочетания из accepted_..., rejected_...
# Каркас: TF-IDF матрица, затем умножение отрицательных признаков(rejected) на -1, затем сжатие матрицы с помощью SVD, и дальнейшее обучение Random Forest.

In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
test=pd.read_pickle("test_df.pkl")
test.drop(columns=["keywords","html"],inplace=True)

In [3]:
train=pd.read_pickle("train_df.pkl")
train.drop(columns=["keywords","html"],inplace=True)

In [4]:
features_list=['accepted_function','rejected_function','accepted_product','rejected_product']
for df in [train, test]:
    df["text"]=df["text"].apply(lambda x: x[0].lower()+x[1].lower() if (len(x)==2) else x[0].lower())
    for j in features_list:
        df[j]=df[j].apply(lambda x: x.lower())
        df[j]=df[j].apply(lambda x: "" if x=="_nan_" else x)
        df[j]=df[j].apply(lambda x: x.replace(" and ",", "))

In [5]:
train.head()

,id,text,accepted_function,rejected_function,accepted_product,rejected_product,target
0,b811ea46-1bc5-4a18-a64c-7504c382da44,release\n2020\n2019\n2018\n2017\n2020\n2019\n2...,"accounting, bookeeping, auditing, finance, hr,...","manufacture production processing assembly, re...",,,2
1,b5e3f6d4-30bc-443c-a9d6-8ed41d5c33b3,sap integrated business planning (ibp)\nsap ad...,"it consultancy, it support services, it manage...","distribution, sale, trade, wholesale, retail s...",,,2
2,997ae1ab-93c4-45a9-9bd0-a49b2b952f1f,facebook\ntwitter\nlinkedin\ninstagram\nwelcom...,"general accounting, auditing, book keeping act...","manufacture, production, processing, distribut...",,,1
3,6890f2f1-bdc1-48cb-bf7f-00944eb37e77,anderson business travel\nspecialist independe...,"insurance brokerage, insurance agents, travel ...","software development, wholesale, retail sale, ...",,,2
4,74a95f04-e440-4af0-850a-eec5a44ba397,amari ireland ltd\nstandards\npeople\nquality\...,"sales agent, sales representative, product bro...","manufacture, production, processing, retail, o...","product, motor vehicles, buses, trucks, motorc...",,1


In [6]:
words_list={}
for df in [train,test]:
    for i in features_list:
        for k in range(len(train)):
            for j in train[i].iloc[k].split(", "):
                words_list[j]=1

In [7]:
vocab=list(words_list)
vocab.remove('')

In [8]:
vocab

['accounting',
 'bookeeping',
 'auditing',
 'finance',
 'hr',
 'training',
 'recruitment',
 'legal',
 'it support services',
 'marketing',
 'public relations',
 'management consultancy',
 'it consultancy',
 'it management',
 'maintenance',
 'ict services',
 'it infrastructure',
 'it managed services',
 'systems integration',
 'systems management',
 'virtualisation',
 'cloud services',
 'information security',
 'database management',
 'software consultancy services',
 'server hosting',
 'data centre',
 'conference call management services',
 'general accounting',
 'book keeping activities',
 'human resource',
 'recruitment agency',
 'training services',
 'it support',
 'managed services',
 'it integration',
 'maintenance services',
 'customization',
 'implementation',
 'installation',
 'it systems management',
 'call centre services',
 'legal activities',
 'business',
 'public opinion polling',
 'market research services',
 'insurance brokerage',
 'insurance agents',
 'travel operator',

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(analyzer="word",stop_words='english',vocabulary=vocab)
mat_train=tfidf.fit_transform(train['text'])
mat_test=tfidf.fit_transform(test['text'])

In [10]:
import scipy
mat_train=scipy.sparse.lil_matrix(mat_train)
mat_train

<15515x1656 sparse matrix of type '<class 'numpy.float64'>'
	with 320623 stored elements in List of Lists format>

In [11]:
mat_test=scipy.sparse.lil_matrix(mat_test)

In [12]:
for j in range(len(train)):
    temp1=train['rejected_function'].iloc[j]
    temp2=train['rejected_product'].iloc[j]
    for i in range(len(vocab)):
        if(temp2.find(vocab[i])!=-1):
            mat_train[j,i]=(-1)*mat_train[j,i]
        elif(temp1.find(vocab[i])!=-1):
            mat_train[j,i]=(-3)*mat_train[j,i]
for j in range(len(test)):
    temp1=test['rejected_function'].iloc[j]
    temp2=test['rejected_product'].iloc[j]
    for i in range(len(vocab)):
        if(temp2.find(vocab[i])!=-1):
            mat_test[j,i]=(-1)*mat_test[j,i]
        elif(temp1.find(vocab[i])!=-1):
            mat_test[j,i]=(-3)*mat_test[j,i]

In [13]:
from sklearn.decomposition import TruncatedSVD,PCA
svd=TruncatedSVD(n_components=100)
#svd=PCA(n_components=100)
#mat_train=mat_train.toarray()
#mat_test=mat_test.toarray()
mat_train=svd.fit_transform(mat_train)
mat_test=svd.transform(mat_test)

In [14]:
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier, BaggingClassifier, VotingClassifier
from sklearn.linear_model import *
clf=RandomForestClassifier(n_estimators=1000,n_jobs=-1)
#clf=ExtraTreesClassifier(n_jobs=-1)
#clf=VotingClassifier(base_estimator=LogisticRegression(),n_estimators=20,n_jobs=-1)
y_train=np.array(train['target'])
clf.fit(mat_train,y_train)
result=clf.predict(mat_test)

In [15]:
df=pd.DataFrame(data=np.hstack([np.array(test['id']).reshape(len(test),1),result.reshape(len(test),1)]),columns=['id','target'])
df.to_csv("result.csv",index=False)

# Что дает следующие результаты на Kaggle:
# public sample: 0.699
# private sample: 0.694
